In [9]:
import os
if not os.path.exists('dataset-food17'):
    !gsutil cp -R gs://foodclassify-6009614683/dataset-food17 /home/jupyter

In [10]:
!pwd

/home/jupyter


In [11]:
!ls

dataset-food17	foodclassify.ipynb  src  tutorials


In [12]:
import matplotlib.pyplot as plt
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from torch.utils.data import DataLoader as dalo
from torch.utils.data import random_split
from torch.utils.data import Dataset
import torchvision.datasets as datasets
from torchvision import transforms
import pandas as pd
from sklearn.model_selection import GridSearchCV

In [13]:
def load_data(path='',transform=None):
    train_dataset = datasets.ImageFolder(path+'/train', transform=transform)
    test_dataset = datasets.ImageFolder(path+'/test', transform=transform)
    data = [dalo(train_dataset), dalo(test_dataset)]
    imgs = []
    labels = []
    for d in data:
      for img, label in d:
        imgs.append(img.numpy().reshape((-1, 128*128)).squeeze())
        labels.append(label.squeeze().numpy())
    return np.array(imgs), np.array(labels)

In [14]:
tran = transforms.Compose( [transforms.Resize((128, 128)),
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                            std=[0.229, 0.224, 0.225]),
                        transforms.Grayscale(num_output_channels=1)]
                        )
img, label = load_data('./dataset-food17',transform=tran)

In [15]:
print(img.shape)
print(label.shape)

X_train, X_test, y_train, y_test = train_test_split(
    img, label, test_size=0.2, shuffle=True)
param_grid={'C':[0.1,1,10,100],'gamma':[0.0001,0.001,0.1,1],'kernel':['rbf','poly']}
clf = GridSearchCV(svm.SVC(), param_grid=param_grid, return_train_score=False)
print("Im Learning.....")
clf.fit(X_train, y_train)
clf.cv_results_
df = pd.DataFrame(clf.cv_results_)
df.to_csv('./result_svm.csv')
print(df)
print(df[['param_C','param_kernel','mean_test_score']])
print(clf.best_params_)
print(clf.best_score_)

(3447, 16384)
(3447,)
Im Learning.....
    mean_fit_time  std_fit_time  mean_score_time  std_score_time param_C  \
0       61.446670      0.708841        27.148426        0.124489     0.1   
1       58.064754      0.657857         9.961204        0.018062     0.1   
2       62.068787      0.386485        27.568868        0.387726     0.1   
3       58.977511      1.143177         9.927604        0.034849     0.1   
4       60.031912      0.833818        26.502309        0.112695     0.1   
5       58.027904      0.820000         9.936770        0.021973     0.1   
6       61.220692      0.291594        26.804917        0.110145     0.1   
7       59.655621      1.011693         9.937171        0.011131     0.1   
8       60.741145      0.352391        26.839002        0.024596       1   
9       59.745942      0.226678         9.927531        0.015885       1   
10      61.952606      0.201805        26.912478        0.091345       1   
11      59.802326      0.398891         9.945916 